# 🏗️ Procesador Sísmico - Análisis de Daño Estructural

## 📋 Descripción
Este cuaderno ejecuta el análisis completo de daño estructural por sismos, replicando las macros VBA originales:
- **Hinges_List**: Identificación y procesamiento de rótulas plásticas
- **Moment_Rotation**: Análisis momento-rotación e histéresis
- **Damage_Index**: Evaluación de índices de daño según Park & Ang

### 🔬 Metodología
- Basado en Jiang, H.J., Chen, L.Z. & Chen, Q. (2011)
  

---

## 🔧 Configuración Inicial

In [ ]:
#!/usr/bin/env python3
"""
Procesador Sísmico - Análisis de Daño Estructural
Conversión completa de macros VBA a Python
"""

import logging
import os
from datetime import datetime
import pandas as pd
import helpers.processor_helper
from macros.hinges_list import macro_hinges_list
from macros.moment_rotation import macro_moment_rotation
from macros.damage_index import macro_damage_index

print("✅ Librerías importadas correctamente")

ModuleNotFoundError: No module named 'procesador_sismico_limpio'

## ⚙️ Parámetros de Análisis

In [ ]:
# 📊 PARÁMETROS DE ANÁLISIS
HP = 3.0          # Altura de piso en metros
DIRECCION = 'X'   # Dirección del sismo ('X' o 'Y')

print(f"📋 Parámetros configurados:")
print(f"   🏢 Altura de piso: {HP} m")
print(f"   🌊 Dirección sísmica: {DIRECCION}")
print(f"   📅 Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

📋 Parámetros configurados:
   🏢 Altura de piso: 3.0 m
   🌊 Dirección sísmica: X
   📅 Timestamp: 2025-09-08 11:04:03


## 🗂️ Configuración de Logging y Resultados

In [ ]:
def configurar_logging():
    """Configura el sistema de logging"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_file = f"analisis_sismico_{timestamp}.log"
    
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file, encoding='utf-8'),
            logging.StreamHandler()
        ]
    )
    return log_file

def crear_directorio_resultados():
    """Crea directorio para resultados con timestamp"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    directorio = f"resultados_{timestamp}"
    os.makedirs(directorio, exist_ok=True)
    return directorio

def guardar_csv_formato_europeo(df, archivo, columnas_numericas=None):
    """Guarda DataFrame en formato CSV europeo (separador ; y coma decimal)"""
    df_export = df.copy()
    
    if columnas_numericas:
        for col in columnas_numericas:
            if col in df_export.columns:
                df_export[col] = df_export[col].astype(str).str.replace('.', ',')
    
    df_export.to_csv(archivo, sep=';', index=False, encoding='utf-8')

# Configurar logging y directorio
log_file = configurar_logging()
dir_resultados = crear_directorio_resultados()
logger = logging.getLogger(__name__)

print(f"📝 Log configurado: {log_file}")
print(f"📁 Directorio de resultados: {dir_resultados}")

📝 Log configurado: analisis_sismico_20250908_110403.log
📁 Directorio de resultados: resultados_20250908_110403


## 📂 PASO 1: Carga de Archivos de Entrada

In [ ]:
logger.info("="*60)
logger.info("🏗️  PROCESADOR SÍSMICO - ANÁLISIS DE DAÑO ESTRUCTURAL")
logger.info("="*60)
logger.info(f"📋 Parámetros: Hp={HP}m, Dirección={DIRECCION}")
logger.info(f"📝 Log guardado en: {log_file}")
logger.info(f"📁 Directorio de resultados: {dir_resultados}")

print("\n🔄 PASO 1: Cargando archivos de entrada...")
logger.info("\n🔄 PASO 1: Cargando archivos de entrada...")

try:
    archivos = cargar_archivos()
    
    print("✅ Archivos cargados exitosamente:")
    for nombre, df in archivos.items():
        print(f"   📄 {nombre}.csv: {len(df)} filas, {len(df.columns)} columnas")
        
except Exception as e:
    print(f"❌ Error cargando archivos: {e}")
    logger.error(f"Error cargando archivos: {e}")
    raise

2025-09-08 11:04:03,579 - INFO - ============================================================
2025-09-08 11:04:03,588 - INFO - 🏗️  PROCESADOR SÍSMICO - ANÁLISIS DE DAÑO ESTRUCTURAL
2025-09-08 11:04:03,589 - INFO - ============================================================
2025-09-08 11:04:03,591 - INFO - 📋 Parámetros: Hp=3.0m, Dirección=X
2025-09-08 11:04:03,592 - INFO - 📝 Log guardado en: analisis_sismico_20250908_110403.log
2025-09-08 11:04:03,593 - INFO - 📁 Directorio de resultados: resultados_20250908_110403
2025-09-08 11:04:03,594 - INFO - 
🔄 PASO 1: Cargando archivos de entrada...



🔄 PASO 1: Cargando archivos de entrada...


/Users/saza/Documents/python project/Andres/procesador_csv/procesador_sismico_limpio.py:32: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,13,14,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  archivos['CR'] = pd.read_csv('csv/CR.csv', sep=';')


✅ Archivos cargados exitosamente:
   📄 CD.csv: 556 filas, 9 columnas
   📄 CR.csv: 1048573 filas, 23 columnas
   📄 HK.csv: 1793 filas, 3 columnas
   📄 SC.csv: 4 filas, 45 columnas


## 🔗 PASO 2: Macro Hinges_List - Identificación de Rótulas

In [ ]:
print("\n🔄 PASO 2: Ejecutando Hinges_List...")
logger.info("\n🔄 PASO 2: Ejecutando Hinges_List...")

try:
    df_rt = macro_hinges_list(archivos, hp=HP, direccion=DIRECCION)
    
    print(f"✅ Hinges_List completado:")
    print(f"   🔗 Rótulas identificadas: {len(df_rt)}")
    print(f"   📊 Columnas generadas: {len(df_rt.columns)}")
    
    # Mostrar primeras 5 rótulas
    print("\n📋 Primeras 5 rótulas procesadas:")
    display(df_rt[['Hinge', 'Section', 'Frame', 'My', 'Mu', 'Cy', 'Cu']].head())
    
except Exception as e:
    print(f"❌ Error en Hinges_List: {e}")
    logger.error(f"Error en Hinges_List: {e}")
    raise

2025-09-08 11:04:04,708 - INFO - 
🔄 PASO 2: Ejecutando Hinges_List...



🔄 PASO 2: Ejecutando Hinges_List...
Ejecutando Hinges_List (Hp=3.0m, Dirección=X)
Identificadas 208 rótulas únicas
RT generado con 208 rótulas
✅ Hinges_List completado:
   🔗 Rótulas identificadas: 208
   📊 Columnas generadas: 19

📋 Primeras 5 rótulas procesadas:


,Hinge,Section,Frame,My,Mu,Cy,Cu
0,44H2,C1B,44.0,654.195512,800.745470,0.010562,0.238827
1,44H1,C1B,44.0,655.484761,801.160521,0.010559,0.237893
2,45H2,C1B,45.0,640.779994,796.426571,0.010597,0.248542
3,45H1,C1B,45.0,642.069157,796.841595,0.010594,0.247608
4,48H2,C1A,48.0,981.200873,1142.645876,0.007449,0.125370


## 💾 Guardar RT.csv (Resultados de Rótulas)

In [ ]:
# Guardar RT.csv
archivo_rt = os.path.join(dir_resultados, 'RT.csv')
columnas_numericas_rt = ['RelDist', 'L', 'Storey', 'B', 'H', "f'c", 'fy', 'ρsx', 'α', 
                        'P average', 'My', 'Mu', 'Cy', 'Cu', 'Lc*', 'Rp']

guardar_csv_formato_europeo(df_rt, archivo_rt, columnas_numericas_rt)
logger.info(f"💾 RT.csv guardado: {archivo_rt}")
print(f"💾 RT.csv guardado: {archivo_rt}")
print(f"   📏 Tamaño: {os.path.getsize(archivo_rt):,} bytes")

2025-09-08 11:05:05,674 - INFO - 💾 RT.csv guardado: resultados_20250908_110403/RT.csv


💾 RT.csv guardado: resultados_20250908_110403/RT.csv
   📏 Tamaño: 38,853 bytes


## 🔄 PASO 3: Macro Moment_Rotation - Análisis Momento-Rotación

In [ ]:
print("\n🔄 PASO 3: Ejecutando Moment_Rotation...")
logger.info("\n🔄 PASO 3: Ejecutando Moment_Rotation...")

try:
    mr_matrix_data, df_rt_updated = macro_moment_rotation(archivos, df_rt, direccion=DIRECCION)
    
    print(f"✅ Moment_Rotation completado:")
    print(f"   📊 Rótulas procesadas: {len(df_rt_updated)}")
    print(f"   🔢 Matrices M-R generadas: {len(mr_matrix_data)}")
    
    # Mostrar estadísticas de rotaciones
    rotaciones_cols = ['θy', 'θp', 'θu', 'θc', 'θm']
    disponibles = [col for col in rotaciones_cols if col in df_rt_updated.columns]
    
    if disponibles:
        print("\n📈 Estadísticas de rotaciones:")
        display(df_rt_updated[disponibles].describe())
    
except Exception as e:
    print(f"❌ Error en Moment_Rotation: {e}")
    logger.error(f"Error en Moment_Rotation: {e}")
    raise

2025-09-08 11:05:05,681 - INFO - 
🔄 PASO 3: Ejecutando Moment_Rotation...



🔄 PASO 3: Ejecutando Moment_Rotation...
Ejecutando Moment_Rotation (Dirección=X)
Moment_Rotation completado para 208 rótulas
✅ Moment_Rotation completado:
   📊 Rótulas procesadas: 208
   🔢 Matrices M-R generadas: 208

📈 Estadísticas de rotaciones:


,θy,θp,θu,θc,θm
count,208.000000,208.000000,208.000000,208.000000,2.080000e+02
mean,0.000428,0.053032,0.053460,0.000053,4.445539e-05
std,0.000148,0.022524,0.022378,0.000002,1.083881e-04
min,0.000279,0.026020,0.026653,0.000051,5.443524e-08
25%,0.000293,0.027872,0.028477,0.000051,1.134229e-05
50%,0.000313,0.069713,0.070006,0.000052,1.525811e-05
75%,0.000605,0.075268,0.075551,0.000056,2.595734e-05
max,0.000632,0.077935,0.078229,0.000057,9.500344e-04


## 💾 Guardar MR.csv (Matrices Momento-Rotación)

In [ ]:
def crear_mr_matricial(mr_matrix_data, rotulas_ordenadas, archivo_salida):
    """Crea archivo MR.csv en formato matricial"""
    
    # Crear encabezados
    header_row1 = []
    header_row2 = []
    header_row3 = []
    
    for rotula in rotulas_ordenadas:
        header_row1.extend([rotula, '', ''])
        header_row2.extend(['M', 'Rot', 'P'])
        header_row3.extend(['kN-m', 'Rad', 'kN'])
    
    # Determinar número máximo de filas
    max_rows = 0
    for rotula in rotulas_ordenadas:
        if rotula in mr_matrix_data:
            max_rows = max(max_rows, len(mr_matrix_data[rotula]['moments']))
    
    # Crear filas de datos
    data_rows = []
    for i in range(max_rows):
        row = []
        for rotula in rotulas_ordenadas:
            if rotula in mr_matrix_data and i < len(mr_matrix_data[rotula]['moments']):
                moment = mr_matrix_data[rotula]['moments'][i]
                rotation = mr_matrix_data[rotula]['rotations'][i]
                axial = mr_matrix_data[rotula]['axials'][i]
                
                # Formatear números con coma decimal
                moment_str = f"{moment:.6f}".replace('.', ',')
                rotation_str = f"{rotation:.6f}".replace('.', ',')
                axial_str = f"{axial:.2f}".replace('.', ',')
                
                row.extend([moment_str, rotation_str, axial_str])
            else:
                row.extend(['', '', ''])
        data_rows.append(row)
    
    # Escribir archivo CSV
    with open(archivo_salida, 'w', encoding='utf-8') as f:
        f.write(';'.join(header_row1) + '\n')
        f.write(';'.join(header_row2) + '\n')
        f.write(';'.join(header_row3) + '\n')
        
        for row in data_rows:
            f.write(';'.join(row) + '\n')

## 🎯 PASO 4: Macro Damage_Index - Evaluación de Daño

In [ ]:
logger.info("\n🔄 PASO 4: Ejecutando Damage_Index...")
df_id, df_rt_final = macro_damage_index(df_rt_updated)

# Guardar ID.csv
archivo_id = os.path.join(dir_resultados, 'ID.csv')
columnas_numericas_id = ['EH', 'ID']
guardar_csv_formato_europeo(df_id, archivo_id, columnas_numericas_id)
logger.info(f"💾 ID.csv guardado: {archivo_id}")

2025-09-08 11:05:07,996 - INFO - 
🔄 PASO 4: Ejecutando Damage_Index...
2025-09-08 11:05:08,017 - INFO - 💾 ID.csv guardado: resultados_20250908_110403/ID.csv


Ejecutando Damage_Index
Damage_Index completado para 208 rótulas


## 💾 Guardar ID.csv (Índices de Daño)

In [ ]:
# Guardar ID.csv
archivo_id = os.path.join(dir_resultados, 'ID.csv')
columnas_numericas_id = ['ID', 'no', 'Beta', 'dM']
guardar_csv_formato_europeo(df_id, archivo_id, columnas_numericas_id)
logger.info(f"💾 ID.csv guardado: {archivo_id}")
print(f"💾 ID.csv guardado: {archivo_id}")
print(f"   📏 Tamaño: {os.path.getsize(archivo_id):,} bytes")

2025-09-08 11:05:08,023 - INFO - 💾 ID.csv guardado: resultados_20250908_110403/ID.csv


💾 ID.csv guardado: resultados_20250908_110403/ID.csv
   📏 Tamaño: 9,176 bytes


## 📋 Actualizar RT.csv con Rotaciones Calculadas

In [ ]:
# Actualizar RT.csv con rotaciones calculadas
columnas_numericas_rt_updated = columnas_numericas_rt + ['θy', 'θp', 'θu', 'θc', 'θm', 'Pm', 'EH']
guardar_csv_formato_europeo(df_rt_updated, archivo_rt, columnas_numericas_rt_updated)
logger.info(f"🔄 RT.csv actualizado con rotaciones: {archivo_rt}")
print(f"🔄 RT.csv actualizado con rotaciones calculadas")

2025-09-08 11:05:08,034 - INFO - 🔄 RT.csv actualizado con rotaciones: resultados_20250908_110403/RT.csv


🔄 RT.csv actualizado con rotaciones calculadas


## 🎉 Resumen Final del Análisis

In [ ]:
print("\n" + "="*60)
print("🎉 ANÁLISIS SÍSMICO COMPLETADO EXITOSAMENTE")
print("="*60)

logger.info("\n" + "="*60)
logger.info("🎉 ANÁLISIS SÍSMICO COMPLETADO EXITOSAMENTE")
logger.info("="*60)

# Resumen de archivos generados
archivos_generados = {
    'RT.csv': 'Resultados de rótulas con propiedades y rotaciones',
    'MR.csv': 'Matrices momento-rotación para cada rótula',
    'ID.csv': 'Índices de daño y niveles de desempeño'
}

print(f"\n📁 Directorio de resultados: {dir_resultados}")
print(f"📝 Archivo de log: {log_file}")
print(f"\n📄 Archivos generados:")

for archivo, descripcion in archivos_generados.items():
    ruta_completa = os.path.join(dir_resultados, archivo)
    if os.path.exists(ruta_completa):
        tamaño = os.path.getsize(ruta_completa)
        print(f"   ✅ {archivo}: {descripcion} ({tamaño:,} bytes)")
        logger.info(f"✅ {archivo}: {descripcion} ({tamaño:,} bytes)")
    else:
        print(f"   ❌ {archivo}: No encontrado")
        logger.warning(f"❌ {archivo}: No encontrado")

# Estadísticas finales
print(f"\n📊 Estadísticas del análisis:")
print(f"   🔗 Rótulas procesadas: {len(df_rt_updated)}")
print(f"   🏢 Altura de piso: {HP} m")
print(f"   🌊 Dirección sísmica: {DIRECCION}")

if 'ND' in df_id.columns:
    niveles = df_id['ND'].value_counts()
    print(f"\n🎯 Distribución final de desempeño:")
    for nivel, cantidad in niveles.items():
        porcentaje = cantidad/len(df_id)*100
        print(f"   {nivel} (Nivel {nivel}): {cantidad} rótulas ({porcentaje:.1f}%)")
        logger.info(f"{nivel}: {cantidad} rótulas ({porcentaje:.1f}%)")

logger.info(f"Análisis completado - Resultados en: {dir_resultados}")
logger.info(f"Rótulas procesadas: {len(df_rt_updated)}")

print(f"\n✨ Análisis sísmico finalizado correctamente")
print(f"📂 Revisa los archivos en: {dir_resultados}")

2025-09-08 11:05:08,039 - INFO - 
2025-09-08 11:05:08,039 - INFO - 🎉 ANÁLISIS SÍSMICO COMPLETADO EXITOSAMENTE
2025-09-08 11:05:08,039 - INFO - ============================================================
2025-09-08 11:05:08,040 - INFO - ✅ RT.csv: Resultados de rótulas con propiedades y rotaciones (74,082 bytes)
2025-09-08 11:05:08,040 - WARNING - ❌ MR.csv: No encontrado
2025-09-08 11:05:08,040 - INFO - ✅ ID.csv: Índices de daño y niveles de desempeño (9,176 bytes)
2025-09-08 11:05:08,041 - INFO - Análisis completado - Resultados en: resultados_20250908_110403
2025-09-08 11:05:08,041 - INFO - Rótulas procesadas: 208



🎉 ANÁLISIS SÍSMICO COMPLETADO EXITOSAMENTE

📁 Directorio de resultados: resultados_20250908_110403
📝 Archivo de log: analisis_sismico_20250908_110403.log

📄 Archivos generados:
   ✅ RT.csv: Resultados de rótulas con propiedades y rotaciones (74,082 bytes)
   ❌ MR.csv: No encontrado
   ✅ ID.csv: Índices de daño y niveles de desempeño (9,176 bytes)

📊 Estadísticas del análisis:
   🔗 Rótulas procesadas: 208
   🏢 Altura de piso: 3.0 m
   🌊 Dirección sísmica: X

✨ Análisis sísmico finalizado correctamente
📂 Revisa los archivos en: resultados_20250908_110403
